In [ ]:
import datetime
import json
import logging
import re
import sys
import urllib.request as ur
import urllib.parse as par


def main():
    url= "https://www.openhumans.org/api/public-data/?source=american_gut"
    html = ur.urlopen(url).read()
    print(type(html))
    data = json.loads(html.decode())
    print(type(data))
    with open ('testfile.json', mode='w') as outfile:
        json.dump(data, outfile)

if __name__ == '__main__':
    main()
    

<class 'bytes'>
<class 'dict'>


In [2]:
import json
import urllib.request as ur
from pprint import pprint
from urllib.request import urlopen
import csv

list_terms = ['diabetes', 'alzheimers','autoimmune' ]

url= "https://www.openhumans.org/api/public-data/?source=american_gut"
site = ur.urlopen(url).read()
data = json.loads(site.decode())

outfile = open('AmericanGut.txt', mode = 'a')
    
#als de next niet niets is dan gaat de loop eeuwig verder zo itereer je over heel de site
while data['next'] != None:
    for i in range(0,len(data["results"]),1):
        #zoekt naar metadata.json files en gaat deze dan de gegevens ophalen
        if 'metadata.json' in data["results"][i]["basename"]:
            username = data["results"][i]["user"]["name"]
            downloadurl = data["results"][i]["download_url"]
            basename = data["results"][i]["basename"]
            print_out = (" username: {} \n url: {} \n basename: {} \n".format(username, downloadurl, basename))
            
            #open de json file en zoek daarin de bepaalde ziekten en rapporteer ze
            link_data = (ur.urlopen(downloadurl).read())
            link = json.loads(link_data.decode())
            for terms in list_terms:
                if link:
                    print_out +=( "\t" + terms+ ':' + link[terms])
                else:
                    print_out += ('Not found')
            print_out += ("\n")
            outfile.write(print_out)
            
    #open de volgende url en laat deze draaien als er over de 100 range is gepasseerd zodat de 2de link kan gestart 
    #worden
    url = data['next']
    site = ur.urlopen(url).read()
    data = json.loads(site.decode())

outfile.close()

In [ ]:
Harvard project

In [3]:
import json
import urllib.request as ur
from pprint import pprint
from urllib.request import urlopen

list_terms = ['Severe disease or rare genetic trait']

url= "https://www.openhumans.org/api/public-data/?source=pgp"
site = ur.urlopen(url).read()
data = json.loads(site.decode())

header = ("username ; url ; basename ;")

output_file = open('Harvard.csv', mode = 'a')

for terms in list_terms:
    header += (terms + ";")
output_file.write(header + "\n")

#als de next niet niets is dan gaat de loop eeuwig verder zo itereer je over heel de site
while data['next'] != None:
    #limit is lengte van de data+ sprongen van 1 om geen gegevens over te slaan
    for i in range(0,len(data["results"]),1):
        #zoekt naar metadata.json files en gaat deze dan de gegevens ophalen
        if 'surveys' and '.json' in data["results"][i]["basename"]:
            username = data["results"][i]["user"]["name"]
            downloadurl = data["results"][i]["download_url"]
            basename = data["results"][i]["basename"]
            print_out = (" {};{};{};".format(username, downloadurl, basename))
            
            #open de json file en zoek daarin de bepaalde ziekten en rapporteer ze
            link_data = (ur.urlopen(downloadurl).read())
            link = json.loads(link_data.decode())
            for j in range(0, len(link)):
                responses = link[j]["responses"]
                for response in responses:
                    nextquery = response["query"]
                    responsevalue = response["response"]
                    if nextquery in list_terms:
                        print_out += (responsevalue + "|")
            print_out += ("\n")
            output_file.write(print_out)
            
    #open de volgende url en laat deze draaien als er over de 100 range is gepasseerd zodat de 2de link kan gestart 
    #worden
    url = data['next']
    site = ur.urlopen(url).read()
    data = json.loads(site.decode())

output_file.close()

In [1]:
import json
import urllib.request as ur
from pprint import pprint
from urllib.request import urlopen

#alles wat ik wil opvragen + de mogelijke antwoorden
list_terms = ['diabetes', 'alzheimers','autoimmune','chickenpox', 'age_years' ]
neg_answers = ['I do not have this condition', 'No']
dnk_answers = ['Unknown', 'Unspecified']
pos_answers = ['yes', 'Yes']

url= "https://www.openhumans.org/api/public-data/?source=american_gut"
site = ur.urlopen(url).read()
data = json.loads(site.decode())

outfile = open('AmericanGut.csv', mode = 'a')

#header info opmaken username, url,... ter navigatie
header = ("username ; url ; basename ;")

#laten loopen zodat er een bepaalde header iedere keer terug komt en de info er rechtstreeks onder komt
for terms in list_terms:
    header += (terms + ";")
outfile.write(header + "\n")
    
#als de next niet niets is dan gaat de loop eeuwig verder zo itereer je over heel de site
while data['next'] != None:
    for i in range(0,len(data["results"]),1):
        #zoekt naar metadata.json files en gaat deze dan de gegevens ophalen
        if 'metadata.json' in data["results"][i]["basename"]:
            username = data["results"][i]["user"]["name"]
            downloadurl = data["results"][i]["download_url"]
            basename = data["results"][i]["basename"]
            print_out = ("{}; {}; {};".format(username, downloadurl, basename))
            
            #open de json file en zoek daarin de bepaalde ziekten en rapporteer ze
            link_data = (ur.urlopen(downloadurl).read())
            link = json.loads(link_data.decode())
            for terms in list_terms:
                if link: #in de downloadjson kijken naar de antwoorden van de vermelde ziekten en omvormen naar binair
                    if link[terms] in neg_answers:
                        print_out +=("0 ;")
                    elif link[terms] in dnk_answers:
                        print_out += ("NA ;")
                    elif link[terms] in pos_answers:
                        print_out += ("1 ;")
                elif link:
                    print_out += (link[terms] + ";")
            print_out += ("\n")
            outfile.write(print_out)
            
    #open de volgende url en laat deze draaien als er over de len(data) range is gepasseerd zodat de volgende link 
    #kan gestart worden
    url = data['next']
    site = ur.urlopen(url).read()
    data = json.loads(site.decode())

outfile.close()

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>